In [1]:
import pickle as pkl
import numpy as np

## Loading data

In [2]:
input_dim = 300

In [3]:
with open('Train_'+str(input_dim)+'.pkl', 'rb') as tr:
    X_train_300 = pkl.load(tr)
tr.close()
X_train_300 = X_train_300.reshape(X_train_300.shape[0], 1, X_train_300.shape[1])
X_train_300.shape

(87844, 1, 300)

In [4]:
with open('Val_'+str(input_dim)+'.pkl', 'rb') as val:
    X_val_300 = pkl.load(val)
val.close()
X_val_300 = X_val_300.reshape(X_val_300.shape[0], 1, X_val_300.shape[1])
X_val_300.shape

(1000, 1, 300)

In [5]:
with open('Test_'+str(input_dim)+'.pkl', 'rb') as test:
    X_test_300 = pkl.load(test)
test.close()
X_test_300 = X_test_300.reshape(X_test_300.shape[0], 1, X_test_300.shape[1])
X_test_300.shape

(1000, 1, 300)

In [6]:
with open('Train_twenty.pkl', 'rb') as tr:
    X_train_20 = pkl.load(tr)
tr.close()

In [7]:
with open('Val_twenty.pkl', 'rb') as val:
    X_val_20 = pkl.load(val)
val.close()

In [8]:
with open('Test_twenty.pkl', 'rb') as test:
    X_test_20 = pkl.load(test)
test.close()

In [9]:
with open('Train_labels.txt', 'r') as l:
    train_labels = l.read().split('\n')
l.close()
len(train_labels)

87844

In [10]:
with open('Val_labels.txt', 'r') as v:
    val_labels = v.read().split('\n')
v.close()
len(val_labels)

1000

In [11]:
with open('Test_labels.txt', 'r') as t:
    test_labels = t.read().split('\n')
t.close()
len(test_labels)

1000

In [12]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(train_labels)

LabelEncoder()

In [13]:
y_train_labels = enc.transform(train_labels)
y_train_labels.shape

(87844,)

In [14]:
y_val_labels = enc.transform(val_labels)
y_val_labels.shape

(1000,)

In [15]:
y_test_labels = enc.transform(test_labels)
y_test_labels.shape

(1000,)

In [27]:
import keras
from keras import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Bidirectional, Input, Concatenate, Masking
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from keras import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import EarlyStopping

In [28]:
y_train = to_categorical(y_train_labels)
y_train.shape

(87844, 10)

In [29]:
y_val = to_categorical(y_val_labels)
y_val.shape

(1000, 10)

In [30]:
y_test = to_categorical(y_test_labels)
y_test.shape

(1000, 10)

In [31]:
from sklearn.utils.class_weight import compute_class_weight

In [32]:
# Instantiate the label encoder
le = LabelEncoder()

# Fit the label encoder to our label series
le.fit(list(y_train_labels))

# Create integer based labels Series
y_integers = le.transform(list(y_train_labels))

In [33]:
class_weights = compute_class_weight('balanced', np.unique(y_train_labels), y_train_labels)
class_weights_dict = dict(zip(le.transform(list(le.classes_)), class_weights))

In [34]:
# Parameters
batch_size = 512
epochs = 100
num_classes = 10

## Keras has its own method for making "ensemble"

In [50]:
inputA = Input(shape=(1, input_dim))
inputB = Input(shape=(20,))

CNN для эмбеддингов используется, в противовес тому, что говорили на собрании, есть статьи с arxiv аж с 2014 года. Данные я разделил.

In [51]:
mask = Masking(mask_value=0., input_shape=(1, input_dim))(inputA)
x = Bidirectional(LSTM(200))(mask) 
x = Dropout(0.2)(x)
x = Model(inputs = inputA, outputs = x)

In [52]:
y = Dense(20, activation="relu")(inputB)
y = Dense(10, activation="relu")(y)
y = Model(inputs=inputB, outputs = y)

In [53]:
combined = Concatenate()([x.output, y.output])

In [54]:
z = Dense(200, activation='relu')(combined)
z = Dense(100, activation='relu')(z)
z = Dense(num_classes, activation='softmax')(z)

In [55]:
model = Model(inputs =[x.input, y.input], outputs = z)

In [56]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=[metrics.categorical_accuracy])
print(model.summary())

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
masking_3 (Masking)             (None, 1, 300)       0           input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 400)          801600      masking_3[0][0]                  
____________________________________________________________________________________________

In [57]:
earlystop = EarlyStopping(monitor='val_categorical_accuracy', min_delta = 0.0001, patience=5, verbose=1, mode='auto')
callbacks_list = [earlystop]

In [58]:
model.fit([X_train_300, X_train_20], y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list,
          validation_data=([X_val_300, X_val_20], y_val), class_weight = class_weights_dict, verbose=1)

Train on 87844 samples, validate on 1000 samples
Epoch 1/100
87844/87844 [==============================] - 24s 273us/step - loss: 1.3436 - categorical_accuracy: 0.6091 - val_loss: 0.9043 - val_categorical_accuracy: 0.7070
Epoch 2/100
87844/87844 [==============================] - 21s 235us/step - loss: 0.8480 - categorical_accuracy: 0.7490 - val_loss: 0.7870 - val_categorical_accuracy: 0.7590
Epoch 3/100
87844/87844 [==============================] - 21s 236us/step - loss: 0.7848 - categorical_accuracy: 0.7634 - val_loss: 0.7529 - val_categorical_accuracy: 0.7620
Epoch 4/100
87844/87844 [==============================] - 21s 234us/step - loss: 0.7539 - categorical_accuracy: 0.7735 - val_loss: 0.7529 - val_categorical_accuracy: 0.7740
Epoch 5/100
87844/87844 [==============================] - 20s 233us/step - loss: 0.7342 - categorical_accuracy: 0.7790 - val_loss: 0.7592 - val_categorical_accuracy: 0.7680
Epoch 6/100
87844/87844 [==============================] - 21s 235us/step - loss:

In [59]:
scores = model.evaluate([X_val_300, X_val_20], y_val, verbose=0)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

Val loss: 0.6813932309150695
Val accuracy: 0.789


In [60]:
y_pred = model.predict([X_val_300, X_val_20])
y_pred_classes = [[1 if c == max(a) else 0 for c in a] for a in y_pred]
y_pred_labels = [a.index(1) for a in y_pred_classes]
confusion_matrix(y_pred_labels, y_val_labels)
#y_val_labels

array([[69,  4,  2,  3,  0,  0,  4,  5,  0,  2],
       [ 1, 78,  3,  0,  1,  1,  2,  1,  2,  5],
       [ 7,  4, 84,  4,  1,  1,  0,  3,  1,  1],
       [ 8,  3,  0, 83,  0,  0,  4,  4,  0,  1],
       [ 0,  0,  2,  2, 86,  2,  1,  1,  9,  7],
       [ 6,  1,  1,  1,  3, 86,  6,  1,  0,  4],
       [ 5,  3,  0,  1,  1,  3, 79,  4,  2,  2],
       [ 1,  3,  2,  2,  0,  1,  1, 77,  2,  1],
       [ 0,  2,  1,  4,  4,  1,  2,  2, 77,  7],
       [ 3,  2,  5,  0,  4,  5,  1,  2,  7, 70]])

Не лучший из полученных результатов, просто я подбирал разные параметры, и тот результат пока при тех же параметрах не воспроизвёлся, видимо, элемент случайности.

In [61]:
test_scores = model.evaluate([X_test_300, X_test_20], y_test, verbose=0)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Test loss: 0.7568139250278473
Test accuracy: 0.766


In [62]:
y_pred = model.predict([X_test_300, X_test_20])
y_pred_classes = [[1 if c == max(a) else 0 for c in a] for a in y_pred]
y_pred_labels = [a.index(1) for a in y_pred_classes]
confusion_matrix(y_pred_labels, y_test_labels)

array([[80,  1,  0,  2,  4,  1,  4,  3,  2,  4],
       [ 2, 74,  2,  2,  0,  0,  0,  4,  6,  5],
       [ 3,  4, 85,  1,  2,  0,  3,  4,  0,  2],
       [ 3,  6,  3, 77,  0,  0,  2,  7,  1,  2],
       [ 1,  2,  1,  2, 83,  0,  0,  0, 13,  9],
       [ 2,  3,  1,  7,  1, 89,  5,  3,  0,  2],
       [ 5,  3,  3,  7,  0,  3, 80,  5,  0,  1],
       [ 0,  3,  2,  2,  0,  3,  2, 65,  4,  2],
       [ 2,  4,  1,  0,  6,  2,  2,  5, 64,  4],
       [ 2,  0,  2,  0,  4,  2,  2,  4, 10, 69]])

In [63]:
print(classification_report(y_pred_labels, y_test_labels))

              precision    recall  f1-score   support

           0       0.80      0.79      0.80       101
           1       0.74      0.78      0.76        95
           2       0.85      0.82      0.83       104
           3       0.77      0.76      0.77       101
           4       0.83      0.75      0.79       111
           5       0.89      0.79      0.84       113
           6       0.80      0.75      0.77       107
           7       0.65      0.78      0.71        83
           8       0.64      0.71      0.67        90
           9       0.69      0.73      0.71        95

    accuracy                           0.77      1000
   macro avg       0.77      0.77      0.76      1000
weighted avg       0.77      0.77      0.77      1000



In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot cnn.png', show_shapes=True, show_layer_names=True)